# supervised machine learning regression project-1 (Linear Regression)

##### by Danial Alyousef

## We will work on:
### Data Overview

### Data Cleaning and Feature Engineering

### Split data using Kfold method to 4 dataset to train and test

### Linear Regression Without additional applications as a baseline to find the best model to predict the price for each car and find its best R2 score

### Applying polynomial features and features scaling

### Apply Linear Regression  with polynomial features 

### Using GridSearchCV we will find the best parameters for:
    1-lasso Regression to find the best R2 score depending on its parameters
    2-Ridge Regression to find the best R2 score depending on its parameters
    3-ElasticNet to find the best R2 score depending on its parameters
### Finding the best model for our dataset between the three models which we mentioned above to predict the price for each car 


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', module='sklearn')
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import r2_score

%matplotlib inline

In [11]:
data = pd.read_csv("data\CarPrice_Assignment.csv")

In [12]:
data.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [13]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [14]:
carbrand=data["CarName"].str.split(' ',expand=True)[0]
carbrand

0      alfa-romero
1      alfa-romero
2      alfa-romero
3             audi
4             audi
          ...     
200          volvo
201          volvo
202          volvo
203          volvo
204          volvo
Name: 0, Length: 205, dtype: object

In [15]:
data["CarBrand"]=carbrand
del data["CarName"]
del data["car_ID"]

In [16]:
data.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarBrand
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [17]:
one_hot_encode_cols = data.dtypes[data.dtypes == np.object]
one_hot_encode_cols = one_hot_encode_cols.index.tolist()

C:\Users\admin\AppData\Local\Temp\ipykernel_12868\3963609396.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  one_hot_encode_cols = data.dtypes[data.dtypes == np.object]


In [18]:
one_hot_encode_cols

['fueltype',
 'aspiration',
 'doornumber',
 'carbody',
 'drivewheel',
 'enginelocation',
 'enginetype',
 'cylindernumber',
 'fuelsystem',
 'CarBrand']

In [19]:
for col in one_hot_encode_cols:
    data[col] = pd.Categorical(data[col])
data = pd.get_dummies(data, columns=one_hot_encode_cols)

In [20]:
data.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,CarBrand_porcshce,CarBrand_porsche,CarBrand_renault,CarBrand_saab,CarBrand_subaru,CarBrand_toyota,CarBrand_toyouta,CarBrand_vokswagen,CarBrand_volkswagen,CarBrand_volvo
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,0,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,0,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,0,0,0,0,0,0,0,0,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,0,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
scores_test=[]
scores_train=[]

lr = LinearRegression()

kf=KFold(n_splits=4, random_state=7218, shuffle=True)

X= data.drop(["price"],axis=1)
y = data["price"]

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    
    lr.fit(X_train, y_train)
        
    y_pred = lr.predict(X_test)
    y_pred_t = lr.predict(X_train)
    
    score_test = r2_score(y_test.values, y_pred)
    score_train = r2_score(y_train.values, y_pred_t)
    
    scores_test.append(score_test)
    scores_train.append(score_train)

print("score for test:",scores_test)
print("mean score for test:",np.mean(scores_test))
print("score for train:",scores_train)
print("mean score for train:",np.mean(scores_train))

score for test: [0.8701485455097748, 0.9221284753975324, 0.8524928462252369, 0.9226639679267974]
mean score for test: 0.8918584587648354
score for train: [0.9761271909832286, 0.9668912771441315, 0.9753409246546858, 0.9702374080262708]
mean score for train: 0.9721492002020791


In [35]:
scores_test_pf=[]
scores_train_pf=[]

s=StandardScaler()

kf=KFold(n_splits=4, random_state=7218, shuffle=True)

poly=PolynomialFeatures(degree=2)

lr_pf = LinearRegression()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    

    
    X_train_pf=poly.fit_transform(X_train)
    X_test_pf=poly.fit_transform(X_test)
    X_train_s=s.fit_transform(X_train_pf)
    X_test_s=s.fit_transform(X_test_pf)
    
    lr_pf.fit(X_train_s, y_train)
        
    y_pred = lr_pf.predict(X_test_s)
    y_pred_t = lr_pf.predict(X_train_s)
    
    score_test_pf = r2_score(y_test.values, y_pred)
    score_train_pf = r2_score(y_train.values, y_pred_t)
    
    scores_test_pf.append(score_test_pf)
    scores_train_pf.append(score_train_pf)

print("score for test with PF:",scores_test_pf)
print("mean score for test with PF:",np.mean(scores_test_pf))
print("score for train with PF:",scores_train_pf)
print("mean score for train with PF:",np.mean(scores_train_pf))

score for test with PF: [-1.0804374241476987e+18, -1.2407841290191686e+19, -1.8593562755699717e+22, -5.325981597661762e+22]
mean score for test with PF: -1.7966716752757919e+22
score for train with PF: [0.9998046424922944, 0.9993068667757476, 0.9983723965600056, 0.9984699069112501]
mean score for train with PF: 0.9989884531848244


In [24]:
from sklearn.model_selection import GridSearchCV

estimator = Pipeline([("polynomial_features", PolynomialFeatures(degree=2, include_bias=False,)),
                      ("scaler", StandardScaler()),
                      ("lasso_regression", Lasso(max_iter=10000))])

params = { 'lasso_regression__alpha': np.geomspace(95, 105, 10) }

grid = GridSearchCV(estimator, params, cv=kf)

grid.fit(X, y)

grid.best_score_, grid.best_params_

(0.9117196095630578, {'lasso_regression__alpha': 100.4317922511906})

In [25]:
estimator = Pipeline([("polynomial_features", PolynomialFeatures(degree=2)),
                      ("scaler", StandardScaler()),
                      ("ridge_regression", Ridge())])

params = { 'ridge_regression__alpha': np.geomspace(130, 140, 10) }

grid = GridSearchCV(estimator, params, cv=kf)

grid.fit(X, y)

grid.best_score_, grid.best_params_

(0.9111094362968895, {'ridge_regression__alpha': 130.0})

In [32]:
estimator = Pipeline([("polynomial_features", PolynomialFeatures(degree=2)),
                      ("scaler", StandardScaler()),
                      ("ElasticNet", ElasticNet(max_iter=10000))])

params = {
    'ElasticNet__alpha': np.geomspace(130, 140, 10),
    'ElasticNet__l1_ratio':np.geomspace(0.01, 1, 10),
         }

grid = GridSearchCV(estimator, params, cv=kf)

grid.fit(X, y)

grid.best_score_, grid.best_params_


(0.9099361341311102, {'ElasticNet__alpha': 130.0, 'ElasticNet__l1_ratio': 1.0})

In [26]:
scores_test_las=[]
scores_train_las=[]

s=StandardScaler()

kf=KFold(n_splits=4, random_state=7218, shuffle=True)

poly=PolynomialFeatures(degree=2)

las=Lasso(alpha = 100.4317922511906,max_iter=100000)

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    

    
    X_train_pf=poly.fit_transform(X_train)
    X_test_pf=poly.fit_transform(X_test)
    X_train_s=s.fit_transform(X_train_pf)
    X_test_s=s.fit_transform(X_test_pf)
    
    las.fit(X_train_s, y_train)
        
    y_pred = las.predict(X_test_s)
    y_pred_t = las.predict(X_train_s)
    
    score_test_las = r2_score(y_test.values, y_pred)
    score_train_las = r2_score(y_train.values, y_pred_t)
    
    scores_test_las.append(score_test_las)
    scores_train_las.append(score_train_las)

print("score for test with lasso:",scores_test_las)
print("mean score for test with lasso:",np.mean(scores_test_las))
print("score for train with lasso:",scores_train_las)
print("mean score for train with lasso:",np.mean(scores_train_las))

score for test with lasso: [0.8266315498890824, 0.9212895477773247, 0.7691712313038199, 0.9395858038089642]
mean score for test with lasso: 0.8641695331947978
score for train with lasso: [0.9864657988753152, 0.9819180536141516, 0.9840666822481952, 0.9818366710041933]
mean score for train with lasso: 0.9835718014354637


In [24]:
scores_test_ridge=[]
scores_train_ridge=[]

s=StandardScaler()

kf=KFold(n_splits=4, random_state=7218, shuffle=True)

poly=PolynomialFeatures(degree=2)

r=Ridge(alpha = 130)

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    

    
    X_train_pf=poly.fit_transform(X_train)
    X_test_pf=poly.fit_transform(X_test)
    X_train_s=s.fit_transform(X_train_pf)
    X_test_s=s.fit_transform(X_test_pf)
    
    r.fit(X_train_s, y_train)
        
    y_pred = r.predict(X_test_s)
    y_pred_t = r.predict(X_train_s)
    
    score_test_ridge = r2_score(y_test.values, y_pred)
    score_train_ridge = r2_score(y_train.values, y_pred_t)
    
    scores_test_ridge.append(score_test_ridge)
    scores_train_ridge.append(score_train_ridge)

print("score for test with ridge:",scores_test_ridge)
print("mean score for test with ridge:",np.mean(scores_test_ridge))
print("score for train with ridge:",scores_train_ridge)
print("mean score for train with ridge:",np.mean(scores_train_ridge))

score for test with ridge: [0.7901483428080671, 0.9422701938679175, 0.7864584202000318, 0.9395552429470276]
mean score for test with ridge: 0.864608049955761
score for train with ridge: [0.9890945329466484, 0.9832821409218315, 0.9871827862230297, 0.9817734712084575]
mean score for train with ridge: 0.9853332328249917


In [28]:
scores_test_ElasticNet=[]
scores_train_ElasticNet=[]

s=StandardScaler()

kf=KFold(n_splits=4, random_state=7218, shuffle=True)

poly=PolynomialFeatures(degree=2)

ENet=ElasticNet(alpha = 130, l1_ratio = 1.0, max_iter=100000)

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    

    
    X_train_pf=poly.fit_transform(X_train)
    X_test_pf=poly.fit_transform(X_test)
    X_train_s=s.fit_transform(X_train_pf)
    X_test_s=s.fit_transform(X_test_pf)
    
    ENet.fit(X_train_s, y_train)
        
    y_pred = ENet.predict(X_test_s)
    y_pred_t = ENet.predict(X_train_s)
    
    score_test_ElasticNet = r2_score(y_test.values, y_pred)
    score_train_ElasticNet = r2_score(y_train.values, y_pred_t)
    
    scores_test_ElasticNet.append(score_test_ElasticNet)
    scores_train_ElasticNet.append(score_train_ElasticNet)

print("score for test with ElasticNet:",scores_test_ElasticNet)
print("mean score for test with ElasticNet:",np.mean(scores_test_ElasticNet))
print("score for train with ElasticNet:",scores_train_ElasticNet)
print("mean score for train with ElasticNet:",np.mean(scores_train_ElasticNet))

score for test with ElasticNet: [0.8272482307242395, 0.9254277027677937, 0.7646028479692115, 0.9360555231326894]
mean score for test with ElasticNet: 0.8633335761484835
score for train with ElasticNet: [0.9835991139794469, 0.9790083665279958, 0.9803961552186766, 0.9789519988776708]
mean score for train with ElasticNet: 0.9804889086509475


In [29]:
r=Ridge(alpha = 130)

poly=PolynomialFeatures(degree=2)

s=StandardScaler()

x_pf=poly.fit_transform(X)
x_pf_s=s.fit_transform(x_pf)

r.fit(x_pf_s,y)

Y_pre = r.predict(x_pf_s)

r2_score(y,Y_pre)

0.9834143240710865